In [1]:
#DATA PREPROCESSING

import json

data = json.load(open('en_de_corpus.json', 'r'))

#Script to remove punctuation, ensure <= max allowed length + padding

import string
import random
import math

#replace punctuation with space, ensure all sentences with  1 space
#also all capital to small 
def replacePunctuation(x):
    puncAndNum = string.punctuation+'01233456789'
    x = x.lower()
    for char in x:
        if char in puncAndNum:
            #print(char)
            x=x.replace(char,' ')
    return ' '.join(x.split())

#max allowed raw length=5, pad to make it 5, allow only equal pairs
tempEn = list(map(replacePunctuation,data['en']))
tempDe = list(map(replacePunctuation,data['de']))
maxRawLength = 5
maxFinalLength = maxRawLength + 2

def wordCount(x):
    return len(x.split())

data_en_processed = list()
data_de_processed = list()
for i in range(len(tempEn)):
    if(wordCount(tempEn[i]) <= maxRawLength and wordCount(tempDe[i]) <= maxRawLength
       and wordCount(tempEn[i])==wordCount(tempDe[i])):
        #if(i%10==0):
        #    print((maxFinalLength -1 - wordCount(tempEn[i])))
        data_en_processed.append('< '+tempEn[i]+' >'*(maxFinalLength -1 - wordCount(tempEn[i])))
        data_de_processed.append('< '+tempDe[i]+' >'*(maxFinalLength -1 - wordCount(tempDe[i])))

print("length after processing : "+str(len(data_en_processed)))  

length after processing : 21680


In [2]:
#pick 5 random samples and display, compare with google translate
import time
from mtranslate import translate


start = math.ceil(random.random()*1000)
for i in range(start,start+5):
    print(data_en_processed[i]+" | "+data_de_processed[i])
    print(translate(data_en_processed[i],'de','en'))
    print()


< be quiet for a moment > | < sei mal einen augenblick still >
Ich bin für einen Augenblick still

< tom was promoted to foreman > | < tom wurde zum steiger befördert >
<Tom wurde zum Vorarbeiter befördert>

< there s a serious problem > | < es besteht ein gravierendes problem >
<Da ​​ist ein ernstes Problem>

< i majored in psychology > > | < mein hauptfach war psychologie > >
Ich habe in Psychologie>>

< he s not ready > > | < er ist nicht bereit > >
<Er ist nicht bereit>>



In [3]:
# Convert to list of lists from list of strings, remove space

data_en_processed_tokens = [s.split() for s in data_en_processed]
data_de_processed_tokens = [s.split() for s in data_de_processed]
print(data_de_processed_tokens[77])
print("****************")

#Lower the size
data_en_processed_tokens = data_en_processed_tokens[:100]
data_de_processed_tokens = data_de_processed_tokens[:100]
data_full = data_en_processed_tokens + data_de_processed_tokens
len(data_full)

data_full = sum(data_full, [])
len(data_full)

print(data_full[0:7])
print(data_full[(len(data_full)//2)+0 :(len(data_full)//2)+7])


['<', 'er', 'unterrichtet', 'arabisch', '>', '>', '>']
****************
['<', 'i', 'took', 'the', 'bus', 'back', '>']
['<', 'ich', 'nahm', 'den', 'bus', 'zurück', '>']


In [4]:
#Partition back into En and De

data_en =  data_full[:(len(data_full)//2)]
data_de =  data_full[(len(data_full)//2):]
print(len(data_en))
print(len(data_de))
data_en.index('tea')

700
700


501

In [6]:
len(data_de)
data_en[0:7]

['<', 'i', 'took', 'the', 'bus', 'back', '>']

In [5]:
tempEnList = list()
for i in range(int(len(data_en)/7)):
    tempEnList.append(data_en[i*7:(i+1)*7])
    
#tempEnList.append(['tea'])

tempDeList = list()
for i in range(int(len(data_de)/7)):
    tempDeList.append(data_de[i*7:(i+1)*7])
    
print(tempEnList[71],tempDeList[71])

['<', 'she', 'was', 'making', 'tea', '>', '>'] ['<', 'sie', 'machte', 'gerade', 'tee', '>', '>']


In [7]:
%whos

Variable                   Type        Data/Info
------------------------------------------------
data                       dict        n=2
data_de                    list        n=700
data_de_processed          list        n=21680
data_de_processed_tokens   list        n=100
data_en                    list        n=700
data_en_processed          list        n=21680
data_en_processed_tokens   list        n=100
data_full                  list        n=1400
i                          int         99
json                       module      <module 'json' from '/usr<...>hon3.5/json/__init__.py'>
math                       module      <module 'math' (built-in)>
maxFinalLength             int         7
maxRawLength               int         5
random                     module      <module 'random' from '/h<...>lib/python3.5/random.py'>
replacePunctuation         function    <function replacePunctuation at 0x7fa26fc3b268>
start                      int         955
string                     mo

In [6]:
print(tempEnList[71])

import gensim, logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


modelEn = gensim.models.Word2Vec(tempEnList,size=100,workers=4,min_count=1)
modelEn.save('english_word2vec_new')

modelDe = gensim.models.Word2Vec(tempDeList,size=100,workers=4,min_count=1)
modelDe.save('german_word2vec_new')

['<', 'she', 'was', 'making', 'tea', '>', '>']


Slow version of gensim.models.doc2vec is being used
2017-04-20 18:47:31,747 : WARNING : Slow version of gensim.models.word2vec is being used
2017-04-20 18:47:31,748 : INFO : collecting all words and their counts
2017-04-20 18:47:31,749 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-04-20 18:47:31,750 : INFO : collected 247 word types from a corpus of 700 raw words and 100 sentences
2017-04-20 18:47:31,752 : INFO : Loading a fresh vocabulary
2017-04-20 18:47:31,754 : INFO : min_count=1 retains 247 unique words (100% of original 247, drops 0)
2017-04-20 18:47:31,755 : INFO : min_count=1 leaves 700 word corpus (100% of original 700, drops 0)
2017-04-20 18:47:31,760 : INFO : deleting the raw counts dictionary of 247 items
2017-04-20 18:47:31,763 : INFO : sample=0.001 downsamples 52 most-common words
2017-04-20 18:47:31,765 : INFO : downsampling leaves estimated 334 word corpus (47.8% of prior 700)
2017-04-20 18:47:31,768 : INFO : estimated required memory f

In [9]:
modelEn['tea']

array([ -4.73628519e-03,  -4.73593874e-03,   7.35284062e-04,
         7.66898505e-04,  -1.28498510e-03,   2.13941606e-03,
        -4.58152359e-03,   1.70954247e-03,   1.38603931e-03,
         3.60729755e-03,  -4.26432071e-03,   6.87931024e-04,
         3.07859760e-03,  -4.23170300e-03,  -1.36448292e-03,
         1.04446197e-03,  -3.20789032e-03,  -2.51956470e-03,
        -2.77759484e-03,  -1.66425923e-06,  -2.24990561e-03,
        -3.00988834e-03,  -1.97166315e-04,  -2.90036923e-03,
         1.68665277e-03,  -3.56142945e-03,  -3.81568680e-03,
         3.11863405e-04,   3.31604038e-03,  -9.84616112e-04,
         2.00261944e-03,  -1.71005900e-03,  -1.13032048e-03,
        -8.25506810e-04,   3.23621114e-03,  -4.48995829e-03,
        -4.54470562e-03,  -3.40136141e-03,   4.76405164e-03,
        -2.40558106e-03,  -1.42524403e-03,  -4.96013975e-03,
        -2.17804196e-03,   3.04285972e-03,   2.52219895e-03,
        -2.67080939e-03,   6.66689244e-04,  -5.62526344e-04,
        -1.45264866e-03,

In [10]:
#data_en = [ 'i' if (e == '<' or e == '>') else e for e in data_en ]
#data_de = [ 'ich' if (e == '<' or e == '>') else e for e in data_de ]

In [11]:
#data_en = ['i', 'i', 'took', 'the', 'bus', 'back', 'i', 'i', 'without', 'air', 'we', 'would', 'die', 'i', 'i', 'i', 'study', 'chinese', 'in', 'beijing', 'i']
#data_de = ['ich', 'ich', 'nahm', 'den', 'bus', 'zurück', 'ich', 'ich', 'ohne', 'luft', 'würden', 'wir', 'sterben', 'ich', 'ich', 'ich', 'lerne', 'in', 'peking', 'chinesisch', 'ich']

In [12]:
#print(data_en[21:], data_de[21:])

In [ ]:
import numpy as np
from mtranslate import translate
from pprint import pprint


# data I/O
#data = open('input_en_de_1000.txt', 'r').read() # should be simple plain text file
#data = data_full
chars_en = list(set(data_en))
chars_de = list(set(data_de))
vocab_size = 100

print('data has %d en_chars, %d de_chars, %d vec size.' % (len(data_en), len(data_de), vocab_size))

char_en_to_vec = { ch:modelEn[ch] for ch in chars_en }
char_de_to_vec = { ch:modelDe[ch] for ch in chars_de }

#vec_en_to_char = {  }
#vec_de_to_char = {  }

#char_to_ix = { ch:i for i,ch in enumerate(chars) }
#ix_to_char = { i:ch for i,ch in enumerate(chars) }

# hyperparameters---> 43     loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)

hidden_size = 150 # size of hidden layer of neurons
seq_length = 7 # number of steps to unroll the RNN for
learning_rate = 1e-2 #1e-1

#globals
#en_ch_cnt = (len(data_full)//2) #len(data_full)//2
test_sentence = ['<', 'i', 'took', 'the', 'bus', 'back', '>']
#test_sentence = [ '<', 'the', 'boy', 'is', 'wearing', 'glasses', '>']
#test_sentence = [ '<', 'i', 'ate', 'a', 'delicious', 'apple', '>']
test_sentence_de= translate(' '.join(test_sentence),'de', 'en').split()

test_sentence_vec = [char_en_to_vec[ch] for ch in test_sentence]
#'<', 'ich', 'nahm', 'den', 'bus', 'zurück', '>'

# model parameters
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1)) # output bias

def lossFun(inputs, targets, hprev):
  """
  inputs,targets are both list of integers.
  hprev is Hx1 array of initial hidden state
  returns the loss, gradients on model parameters, and last hidden state
  """
  xs, hs, ys, ps = {}, {}, {}, {}
  hs[-1] = np.copy(hprev)
  loss = 0
  # forward pass
  for t in range(len(inputs)):
    #xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation
    #xs[t][inputs[t]] = 1
    xs[t] = np.reshape(np.array(inputs[t]), (vocab_size,1) )
    hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state
    ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
    #ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars
    

    #loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)
    
    loss += 0.5*np.square(np.subtract(np.reshape(np.array(targets[t]),(vocab_size,1)) , ys[t]))
    
  # backward pass: compute gradients going backwards
  dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
  dbh, dby = np.zeros_like(bh), np.zeros_like(by)
  dhnext = np.zeros_like(hs[0])
  for t in reversed(range(len(inputs))):
    #dy = np.copy(ps[t])
    #dy[targets[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
    dy = -1*np.subtract(np.reshape(np.array(targets[t]),(vocab_size,1)) , ys[t])
    
    dWhy += np.dot(dy, hs[t].T)
    dby += dy
    dh = np.dot(Why.T, dy) + dhnext # backprop into h
    dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
    dbh += dhraw
    dWxh += np.dot(dhraw, xs[t].T)
    dWhh += np.dot(dhraw, hs[t-1].T)
    dhnext = np.dot(Whh.T, dhraw)
  for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
    np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
  return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]


def sample(h, n):

  n = len(test_sentence_vec)

  translated_vecs = []
  for t in range(n):
    #x = np.zeros((vocab_size, 1))
    #x[char_to_ix[test_sentence[t]]] = 1
    x = np.reshape(np.array(test_sentence_vec[t]), (vocab_size,1) )
        
    h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
    y = np.dot(Why, h) + by   
    translated_vecs.append(y)
  return translated_vecs






n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0
while True:
  # prepare inputs (we're sweeping from left to right in steps seq_length long)
  if p+seq_length+1 >= len(data_en) or n == 0: 
    hprev = np.zeros((hidden_size,1)) # reset RNN memory
    p = 0 # go from start of data
  inputs = [char_en_to_vec[ch] for ch in data_en[p:p+seq_length]]
  #targets = [char_to_ix[ch] for ch in data[p:p+seq_length]]
  targets = [char_de_to_vec[ch] for ch in data_de[p:p+seq_length]]

  # sample from the model now and then
  if n % 10000 == 0:
    slen = len(test_sentence_vec)
    sample_vecs = sample(hprev, slen)
    #ans = [modelDe.similar_by_vector(v.flatten(), topn=3, restrict_vocab=None)[0][0] for v in sample_vecs]
    '''for a in ans:
        pprint(a[0])
        pprint("***")
    '''
    
    print('---------------')
    print('Input Sentence : '+' '.join(test_sentence))
    print('Expected Translation by Google : '+' '.join(test_sentence_de))
    
    i = 0
    for v in sample_vecs:
        top_n = modelDe.similar_by_vector(v.flatten(), topn=3, restrict_vocab=None)
        print("word "+str(i+1)+" : ",end='')
        for j in range(3):
            print(top_n[j][0],"%.2f" %top_n[j][1] ," , " ,end=' ')
        i=i+1
        print()
    #print(ans)
    #print(translate(' '.join(ans),'en', 'de'))
    print('---------------')
    


  # forward seq_length characters through the net and fetch gradient
  loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
  smooth_loss = smooth_loss * 0.999 + loss * 0.001
  if n % 1000 == 0: print('iter %d, loss: %f' % (n, np.mean(smooth_loss)) ) # print progress
  
  # perform parameter update with Adagrad
  for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby], 
                                [mWxh, mWhh, mWhy, mbh, mby]):
    mem += dparam * dparam
    param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update

  p += seq_length # move data pointer
  n += 1 # iteration counter 


data has 700 en_chars, 700 de_chars, 100 vec size.


2017-04-20 18:47:43,376 : INFO : precomputing L2-norms of word weight vectors


---------------
Input Sentence : < i took the bus back >
Expected Translation by Google : <Ich nahm den Bus zurück>
word 1 : herr 0.27  ,  kinder 0.25  ,  ohne 0.24  ,  
word 2 : gestern 0.30  ,  war 0.26  ,  trägt 0.24  ,  
word 3 : alleine 0.36  ,  botschaft 0.21  ,  lernen 0.19  ,  
word 4 : tot 0.26  ,  höre 0.24  ,  nach 0.19  ,  
word 5 : löwen 0.27  ,  dich 0.22  ,  bei 0.22  ,  
word 6 : kein 0.26  ,  mach 0.21  ,  könnten 0.20  ,  
word 7 : am 0.25  ,  gekleidet 0.23  ,  ohne 0.22  ,  
---------------
iter 0, loss: 32.203955
iter 1000, loss: 11.841259
iter 2000, loss: 4.353987
iter 3000, loss: 1.600950
iter 4000, loss: 0.588671
iter 5000, loss: 0.216460
iter 6000, loss: 0.079599
iter 7000, loss: 0.029276
iter 8000, loss: 0.010772
iter 9000, loss: 0.003968
---------------
Input Sentence : < i took the bus back >
Expected Translation by Google : <Ich nahm den Bus zurück>
word 1 : < 0.99  ,  > 0.50  ,  kinder 0.35  ,  
word 2 : ich 0.99  ,  unterrichtet 0.38  ,  > 0.30  ,  
word 

iter 101000, loss: 0.000007
iter 102000, loss: 0.000007
iter 103000, loss: 0.000007
iter 104000, loss: 0.000007
iter 105000, loss: 0.000007
iter 106000, loss: 0.000007
iter 107000, loss: 0.000007
iter 108000, loss: 0.000007
iter 109000, loss: 0.000007
---------------
Input Sentence : < i took the bus back >
Expected Translation by Google : <Ich nahm den Bus zurück>
word 1 : < 0.95  ,  > 0.47  ,  kinder 0.33  ,  
word 2 : ich 0.96  ,  unterrichtet 0.35  ,  gefällt 0.31  ,  
word 3 : nahm 0.55  ,  begann 0.35  ,  zweierreihen 0.33  ,  
word 4 : den 0.74  ,  einen 0.34  ,  war 0.27  ,  
word 5 : bus 0.81  ,  wo 0.31  ,  salz 0.30  ,  
word 6 : zurück 0.88  ,  wäre 0.34  ,  eltern 0.31  ,  
word 7 : > 0.98  ,  < 0.49  ,  war 0.40  ,  
---------------
iter 110000, loss: 0.000007
iter 111000, loss: 0.000007
iter 112000, loss: 0.000007
iter 113000, loss: 0.000007
iter 114000, loss: 0.000007
iter 115000, loss: 0.000007
iter 116000, loss: 0.000007
iter 117000, loss: 0.000007
iter 118000, loss: 

iter 211000, loss: 0.000007
iter 212000, loss: 0.000007
iter 213000, loss: 0.000007
iter 214000, loss: 0.000007
iter 215000, loss: 0.000007
iter 216000, loss: 0.000007
iter 217000, loss: 0.000007
iter 218000, loss: 0.000007
iter 219000, loss: 0.000007
---------------
Input Sentence : < i took the bus back >
Expected Translation by Google : <Ich nahm den Bus zurück>
word 1 : < 0.98  ,  > 0.47  ,  das 0.39  ,  
word 2 : ich 0.96  ,  unterrichtet 0.35  ,  gestern 0.33  ,  
word 3 : nahm 0.58  ,  wäre 0.40  ,  war 0.35  ,  
word 4 : den 0.70  ,  sofort 0.34  ,  einen 0.31  ,  
word 5 : bus 0.81  ,  wo 0.28  ,  zeitschriften 0.28  ,  
word 6 : zurück 0.82  ,  eltern 0.38  ,  ich 0.33  ,  
word 7 : > 0.98  ,  < 0.48  ,  war 0.40  ,  
---------------
iter 220000, loss: 0.000007
iter 221000, loss: 0.000007
iter 222000, loss: 0.000007
iter 223000, loss: 0.000007
iter 224000, loss: 0.000007
iter 225000, loss: 0.000007
iter 226000, loss: 0.000007
iter 227000, loss: 0.000007
iter 228000, loss: 0.0

In [46]:
modelDe.similar_by_vector(yy, topn=1, restrict_vocab=None) 

[('wunder', 0.9999999403953552)]

In [49]:
len(inputs)

7

In [23]:
chars_de

['geht',
 'schwer',
 'reich',
 '>',
 'haare',
 'lied',
 'wir',
 'das',
 'hat',
 'ich',
 'war',
 'salz',
 'für',
 'zu',
 'geduld',
 'nahm',
 'kennen',
 'ins',
 'bus',
 'mir',
 'der',
 '<',
 'blonde',
 'sterben',
 'peking',
 'glauben',
 'ohne',
 'früh',
 'er',
 'trägt',
 'eine',
 'tom',
 'junge',
 'würden',
 'den',
 'lerne',
 'danke',
 'bett',
 'bitte',
 'brille',
 'chinesisch',
 'zurück',
 'dieses',
 'in',
 'luft',
 'ihre']

In [ ]:
sample_vecs[0].flatten().shape

In [ ]:
test_sentence = ['i', 'i', 'took', 'the', 'bus', 'back', 'i']
test_sentence_vec = [char_en_to_vec[ch] for ch in test_sentence]


In [ ]:
%whos

In [ ]:
Variable                   Type        Data/Info
------------------------------------------------
Whh                        ndarray     50x50: 2500 elems, type `float64`, 20000 bytes
Why                        ndarray     497x50: 24850 elems, type `float64`, 198800 bytes (194.140625 kb)
Wxh                        ndarray     50x497: 24850 elems, type `float64`, 198800 bytes (194.140625 kb)
bh                         ndarray     50x1: 50 elems, type `float64`, 400 bytes
by                         ndarray     497x1: 497 elems, type `float64`, 3976 bytes
char_to_ix                 dict        n=497
chars                      list        n=497
dWhh                       ndarray     50x50: 2500 elems, type `float64`, 20000 bytes
dWhy                       ndarray     497x50: 24850 elems, type `float64`, 198800 bytes (194.140625 kb)
dWxh                       ndarray     50x497: 24850 elems, type `float64`, 198800 bytes (194.140625 kb)
data                       list        n=1400
data_de_processed          list        n=21680
data_de_processed_tokens   list        n=100
data_en_processed          list        n=21680
data_en_processed_tokens   list        n=100
data_full                  list        n=1400
data_size                  int         1400
dbh                        ndarray     50x1: 50 elems, type `float64`, 400 bytes
dby                        ndarray     497x1: 497 elems, type `float64`, 3976 bytes
dparam                     ndarray     497x1: 497 elems, type `float64`, 3976 bytes
en_ch_cnt                  int         700
gensim                     module      <module 'gensim' from '/h<...>ages/gensim/__init__.py'>
hidden_size                int         50
hprev                      ndarray     50x1: 50 elems, type `float64`, 400 bytes
i                          int         259
inputs                     list        n=7
ix_to_char                 dict        n=497
json                       module      <module 'json' from '/hom<...>hon3.5/json/__init__.py'>
learning_rate              float       0.01
loss                       float64     23.793629885
lossFun                    function    <function lossFun at 0x7f61930aeb70>
mWhh                       ndarray     50x50: 2500 elems, type `float64`, 20000 bytes
mWhy                       ndarray     497x50: 24850 elems, type `float64`, 198800 bytes (194.140625 kb)
mWxh                       ndarray     50x497: 24850 elems, type `float64`, 198800 bytes (194.140625 kb)
math                       module      <module 'math' from '/hom<...>3.5/lib-dynload/math.so'>
maxFinalLength             int         7
maxRawLength               int         5
mbh                        ndarray     50x1: 50 elems, type `float64`, 400 bytes
mby                        ndarray     497x1: 497 elems, type `float64`, 3976 bytes
mem                        ndarray     497x1: 497 elems, type `float64`, 3976 bytes
modelEn                    Word2Vec    Word2Vec(vocab=6972, size=100, alpha=0.025)
modelGe                    Word2Vec    Word2Vec(vocab=9279, size=100, alpha=0.025)
n                          int         2459
np                         module      <module 'numpy' from '/ho<...>kages/numpy/__init__.py'>
p                          int         581
param                      ndarray     497x1: 497 elems, type `float64`, 3976 bytes
random                     module      <module 'random' from '/h<...>lib/python3.5/random.py'>
replacePunctuation         function    <function replacePunctuation at 0x7f61b38527b8>
sample                     function    <function sample at 0x7f61c4e2b0d0>
sample_ix                  list        n=7
seq_length                 int         7
slen                       int         7
smooth_loss                float64     21.9345595918
start                      int         255
string                     module      <module 'string' from '/h<...>lib/python3.5/string.py'>
targets                    list        n=7
tempDe                     list        n=132173
tempEn                     list        n=132173
test_sentence              list        n=7
time                       module      <module 'time' (built-in)>
translate                  function    <function translate at 0x7f61b27228c8>
txt                        str         < ich ist ist > > >
vocab_size                 int         497
wordCount                  function    <function wordCount at 0x7f61b2b3d510>
x                          ndarray     100: 100 elems, type `float32`, 400 bytes
y                          ndarray     100: 100 elems, type `float32`, 400 bytes